In [ ]:
#default_exp beamformers

# Beamformers

 > Tensorflow implementations of beamformers
 
Supports working on batches.


In [ ]:
#export
import tensorflow as tf
import tensorflow.linalg as linalg

In [ ]:
#export
def cov_matrix(x):
    """Computes the covariance matrix on signal x"""
    n_chirps = x.shape[-1]
    Rxx = tf.matmul(x, x, adjoint_b=True)
    return Rxx/n_chirps

def forward_backward_avg(Rxx):
    """Backward forward averaging"""
    num_rx = Rxx.shape[-1]
    
    J = tf.reverse(tf.eye(num_rx, dtype=Rxx.dtype), [-1])
    
    R_fb = 0.5 * (Rxx + tf.matmul(
        J, tf.matmul(tf.math.conj(Rxx), J)))
    
    return R_fb

def aoa_capon(x, steering_vec, mu = 1e-7, bottom_center=True):
    """Tensorflow implementation of Capon AoA estimatation"""
    num_rx = x.shape[-2]
    Rxx = cov_matrix(x)
    Rxx = forward_backward_avg(Rxx)
    
    if mu is not None:
        uI = tf.eye(num_rx, dtype=x.dtype) * mu    
        Rxx_inv = linalg.inv(Rxx+uI)
    else:
        Rxx_inv = linalg.inv(Rxx)
    Rxx_inv_a = tf.matmul(Rxx_inv, steering_vec, transpose_b=True)
    
    den = tf.math.reciprocal_no_nan(
        tf.einsum('ij,...ji->...i', tf.math.conj(steering_vec), Rxx_inv_a)
    )
    weights = tf.einsum('bij,bj->bi', Rxx_inv_a, den)
    
    return den, weights

In [ ]:
from mmwave import dsp
import numpy as np

np.random.seed(2021)

In [ ]:
xx = np.random.random((64, 8, 32)) + 1j * np.random.random((64, 8, 32))

npRxx = np.array([dsp.cov_matrix(x) for x in xx])
tfRxx = cov_matrix(xx)

assert np.allclose(npRxx, tfRxx)


In [ ]:
npRxxfb = np.array([dsp.forward_backward_avg(Rxx) for Rxx in npRxx])
tfRxxfb = forward_backward_avg(tfRxx)

assert np.allclose(npRxxfb, tfRxxfb)

In [ ]:
steering_vec = np.random.random((181, 8)) + 1j * np.random.random((181, 8))

npAoA = [dsp.aoa_capon(x, steering_vec) for x in xx]
npSpectrum = np.array([aoa[0] for aoa in npAoA])
npWeights = np.array([aoa[1] for aoa in npAoA])

tfSpectrum, tfWeights = aoa_capon(xx, steering_vec, mu=None)

assert np.allclose(npSpectrum, tfSpectrum)
assert np.allclose(npWeights, tfWeights)

## Example Usage

In [ ]:
#skip
import matplotlib.pyplot as plt
import numpy as np

from radicalsdk.h5dataset import H5DatasetLoader
from radicalsdk.radar.config_v1 import read_radar_params
from radicalsdk.radar.v1 import RadarFrame

data = H5DatasetLoader('../samples/indoor_sample_50.h5')

# Read config and configure RadarFrame object
radar_config = read_radar_params('../samples/indoor_human_rcs.cfg')
rf = RadarFrame(radar_config)

In [ ]:
#skip
# Try on sample data
FRAME_IDX = 2
mmwave_range_az = rf.compute_range_azimuth(data['radar'][FRAME_IDX])
range_cube = rf.range_cube.copy()

Reference implementation in NumPy

In [ ]:
# skip
plt.figure(figsize=(5, 10))
plt.imshow(np.log(np.abs(rf.compute_range_azimuth(data['radar'][FRAME_IDX]))))
plt.show()

Full Tensorflow implementation

In [ ]:
# skip

spectrum, weights = aoa_capon(
    range_cube, 
    steering_vec=tf.cast(rf.steering_vec, tf.complex128),
    mu = None
)

plt.figure(figsize=(5, 10))
plt.imshow(np.fliplr(np.flipud(np.log(np.abs(spectrum)))))
plt.show()